## Alpha by NLP: Getting stock returns from SQLite3 database using Python wrapper function

Let's create a connection and cursor to our SQLite3 database. We can call `cursor.execute()` to perform SQL commands.

In [59]:
import sqlite3
conn = sqlite3.connect('stocks.db')
c = conn.cursor()

sql = "SELECT * FROM stocks WHERE symbol='AAPL' and theDate > strftime('2017-06-01');"

results = []
for row in c.execute(sql):
    results.append([row[0],row[1],row[2],row[8]])

print(results[:10])

[['2017-06-02', 'AAPL', 0.0148191937834, 0.0112729848091], ['2017-06-05', 'AAPL', -0.00977808960652, -0.00899268524104], ['2017-06-06', 'AAPL', 0.00337818504286, 0.00678423926363], ['2017-06-07', 'AAPL', 0.00595660743198, 0.00398528213149], ['2017-06-08', 'AAPL', -0.00244571031878, -0.00297040786488], ['2017-06-09', 'AAPL', -0.038776752088, -0.0371596851367], ['2017-06-12', 'AAPL', -0.0238958121599, -0.0236769405265], ['2017-06-13', 'AAPL', 0.00804564720184, 0.00283575697594], ['2017-06-14', 'AAPL', -0.00975504494863, -0.00840446209595], ['2017-06-15', 'AAPL', -0.00599346222118, -0.00394318735999]]


Now, let's do the same, but put the result in a Pandas dataframe. We will drop the five empty columns in our table.

In [60]:
import pandas as pd

df = pd.read_sql_query(sql, conn)

df.drop('empty1', axis=1, inplace=True)
df.drop('empty2', axis=1, inplace=True)
df.drop('empty3', axis=1, inplace=True)
df.drop('empty4', axis=1, inplace=True)
df.drop('empty5', axis=1, inplace=True)

conn.close()
df[:10]

,theDate,symbol,return,alpha
0,2017-06-02,AAPL,0.014819,0.011273
1,2017-06-05,AAPL,-0.009778,-0.008993
2,2017-06-06,AAPL,0.003378,0.006784
3,2017-06-07,AAPL,0.005957,0.003985
4,2017-06-08,AAPL,-0.002446,-0.002970
5,2017-06-09,AAPL,-0.038777,-0.037160
6,2017-06-12,AAPL,-0.023896,-0.023677
7,2017-06-13,AAPL,0.008046,0.002836
8,2017-06-14,AAPL,-0.009755,-0.008404
9,2017-06-15,AAPL,-0.005993,-0.003943


### Defining the getRets( ) function
Now we're ready to write our getRets function:  
`getRets(symbol, date, horizon)`

We have a SQLite3 database with schema  
`   0       1       2       3       4       5       6       7       8`  
`theDate symbol  return                                          alpha`

`getRet()` returns a list like `[theDate, symbol, return, alpha]`:  
`[('2017-06-01', 'AAPL', 0.00274939783809, -0.00572330324226),
 ('2017-06-02', 'AAPL', 0.0148191937834, 0.0112729848091),
 ('2017-06-05', 'AAPL', -0.00977808960652, -0.00899268524104),
 ('2017-06-06', 'AAPL', 0.00337818504286, 0.00678423926363),
 ('2017-06-07', 'AAPL', 0.00595660743198, 0.00398528213149),
 ('2017-06-08', 'AAPL', -0.00244571031878, -0.00297040786488),
 ('2017-06-09', 'AAPL', -0.038776752088, -0.0371596851367)]`

In [63]:
def getRets(symbol, date, horizon=1):
    conn = sqlite3.connect('stocks.db')
    c = conn.cursor()
    
    results = []
    for row in c.execute("SELECT theDate, Symbol, Return, Alpha \
                         FROM stocks WHERE symbol=? \
                         AND theDate >= strftime(?) \
                         AND theDate < date(strftime(?), ?);", 
                         [symbol, date, date, '+{} day'.format(horizon)]):
        results.append(row)
    return results

In [65]:
getRets('AAPL', '2017-06-01', 10)

[('2017-06-01', 'AAPL', 0.00274939783809, -0.00572330324226),
 ('2017-06-02', 'AAPL', 0.0148191937834, 0.0112729848091),
 ('2017-06-05', 'AAPL', -0.00977808960652, -0.00899268524104),
 ('2017-06-06', 'AAPL', 0.00337818504286, 0.00678423926363),
 ('2017-06-07', 'AAPL', 0.00595660743198, 0.00398528213149),
 ('2017-06-08', 'AAPL', -0.00244571031878, -0.00297040786488),
 ('2017-06-09', 'AAPL', -0.038776752088, -0.0371596851367),
 ('2017-06-01', 'AAPL', 0.00274939783809, -0.00572337451741),
 ('2017-06-02', 'AAPL', 0.0148191937834, 0.0112729549772),
 ('2017-06-05', 'AAPL', -0.00977808960652, -0.00899267863396),
 ('2017-06-06', 'AAPL', 0.00337818504286, 0.00678426791648),
 ('2017-06-07', 'AAPL', 0.00595660743198, 0.00398526554806),
 ('2017-06-08', 'AAPL', -0.00244571031878, -0.00297041227881),
 ('2017-06-09', 'AAPL', -0.038776752088, -0.0371596715334)]

In [66]:
getRets('BBY', '2017-06-01', 10)

[('2017-06-01', 'BBY', 0.015322445114, 0.00645739513555),
 ('2017-06-02', 'BBY', -0.010447777951, -0.0141582023143),
 ('2017-06-05', 'BBY', -0.00251379260981, -0.00169201798006),
 ('2017-06-06', 'BBY', -0.0166330813172, -0.0130693024024),
 ('2017-06-07', 'BBY', 0.0145225015295, 0.0124598890605),
 ('2017-06-08', 'BBY', -0.00454698544043, -0.00509598037693),
 ('2017-06-09', 'BBY', -0.0165792587281, -0.014887310584)]

In [67]:
getRets('MSFT', '2017-06-01', 10)

[('2017-06-01', 'MSFT', 0.00372282381001, -0.00446468904614),
 ('2017-06-02', 'MSFT', 0.0236805142277, 0.0202536690515),
 ('2017-06-05', 'MSFT', 0.00724633480361, 0.00800530270811),
 ('2017-06-06', 'MSFT', 0.00332039296237, 0.00661180060898),
 ('2017-06-07', 'MSFT', -0.00179258143102, -0.0036975525568),
 ('2017-06-08', 'MSFT', -0.00607821530706, -0.00658525168888),
 ('2017-06-09', 'MSFT', -0.0226545805137, -0.0210919433734),
 ('2017-06-01', 'MSFT', 0.00372282381001, -0.00446466835984),
 ('2017-06-02', 'MSFT', 0.0236805142277, 0.0202536777096),
 ('2017-06-05', 'MSFT', 0.00724633480361, 0.00800530079052),
 ('2017-06-06', 'MSFT', 0.00332039296237, 0.00661179229302),
 ('2017-06-07', 'MSFT', -0.00179258143102, -0.00369754774377),
 ('2017-06-08', 'MSFT', -0.00607821530706, -0.00658525040782),
 ('2017-06-09', 'MSFT', -0.0226545805137, -0.0210919473215)]